In [2]:
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.metrics import accuracy_score
import warnings
import matplotlib.pyplot as plt 
from scipy.spatial.distance import cdist
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from tabulate import tabulate 
from sklearn.externals import joblib 
import joblib as job
import pickle as pk
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
#2.1.
from sklearn.model_selection import train_test_split
path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_4/Dataset/yelp_labelled.txt'
r = 0.3

def Split(path,r):
  fp = open(path)
  lines = fp.read()
  lines = lines.split('\n')
  data = []
  for i in range(len(lines)):
    data.append(lines[i].split('\t'))
  Data = pd.DataFrame(data, columns = ['Lines','Label'])  
  Data = Data.dropna()
  Data['Label'] = pd.to_numeric(Data['Label'])
  Data['Lines'] = Data['Lines'].astype(str)
  X = Data['Lines']
  Y = Data['Label'] 
  Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=r,stratify=Y,random_state =3) 
  return Xtrain, Xtest, Ytrain, Ytest

Xtrain, Xtest, Ytrain, Ytest = train_test_split =  Split(path,r)  

In [12]:
print('Xtrain shape :',Xtrain.shape)
print('Ytrain shape :',Ytrain.shape)
print('Xtest shape :',Xtest.shape)
print('Ytest shape :',Ytest.shape)

Xtrain shape : (700,)
Ytrain shape : (700,)
Xtest shape : (300,)
Ytest shape : (300,)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
#2.2.

def RemovePun(Xtrain, Xtest):
  punc = '''!'"\,<()-[]%^&*{};:>./?@#$_~'''
  
  for i in range(len(Xtrain)):
    s1 = Xtrain.iloc[i]
    s2 = ""
    for j in s1:
      if j not in punc:
        s2 = s2+j
    Xtrain.iloc[i] = s2

  for i in range(len(Xtest)):
    s1 = Xtest.iloc[i]
    s2 = ""
    for j in s1:
      if j not in punc:
        s2 = s2+j
    Xtest.iloc[i] = s2  
  
  return Xtrain, Xtest

Xtrain, Xtest = RemovePun(Xtrain, Xtest)
print('\nAfter removing Puncutation,  4 lines of Training Set :\n')
for i in Xtrain[3:7]:
  print(i.split(' ')) 

def LowerCaseWords(Xtrain, Xtest):
  
  for i in range(len(Xtrain)):
    Xtrain.iloc[i] = Xtrain.iloc[i].lower()
 
  for i in range(len(Xtest)):
    Xtest.iloc[i] = Xtest.iloc[i].lower()
  
  return Xtrain, Xtest

Xtrain, Xtest = LowerCaseWords(Xtrain, Xtest)
print('\nAfter lower case , 4 lines of Training Set :\n')
for i in Xtrain[3:7]:
  print(i.split(' ')) 


def RemoveStopWords(Xtrain, Xtest):
  
  stopw = stopwords.words('english')
  NewXtrain =[]
  NewXtest =[]

  for i in range(len(Xtrain)):
    temp = []
    for j in (word_tokenize(Xtrain.iloc[i])): 
      if j not in stopw:
        temp.append(j)
    NewXtrain.append(temp)

  for i in range(len(Xtest)):
    temp = []
    for j in (word_tokenize(Xtest.iloc[i])): 
      if j not in stopw:
        temp.append(j)
    NewXtest.append(temp)

  return NewXtrain ,NewXtest

Xtrain, Xtest =  RemoveStopWords(Xtrain, Xtest)
print('\nAfter removing stopwords , 4 lines of Training Set :\n')
for i in Xtrain[3:7]:
  print(i) 



After removing Puncutation,  4 lines of Training Set :

['first', 'time', 'there', 'and', 'might', 'just', 'be', 'the', 'last']
['Strike', '2', 'who', 'wants', 'to', 'be', 'rushed']
['They', 'were', 'goldencrispy', 'and', 'delicious']
['The', 'servers', 'are', 'not', 'pleasant', 'to', 'deal', 'with', 'and', 'they', 'dont', 'always', 'honor', 'Pizza', 'Hut', 'coupons']

After lower case , 4 lines of Training Set :

['first', 'time', 'there', 'and', 'might', 'just', 'be', 'the', 'last']
['strike', '2', 'who', 'wants', 'to', 'be', 'rushed']
['they', 'were', 'goldencrispy', 'and', 'delicious']
['the', 'servers', 'are', 'not', 'pleasant', 'to', 'deal', 'with', 'and', 'they', 'dont', 'always', 'honor', 'pizza', 'hut', 'coupons']

After removing stopwords , 4 lines of Training Set :

['first', 'time', 'might', 'last']
['strike', '2', 'wants', 'rushed']
['goldencrispy', 'delicious']
['servers', 'pleasant', 'deal', 'dont', 'always', 'honor', 'pizza', 'hut', 'coupons']


In [14]:
#2.3.

def FeatureMatrix(Xtrain, Xtest):
  Vocabword = []
  for i in Xtrain:
    Vocabword =  Vocabword + i
  Vocabword = set(Vocabword)


  NewXtrain = []
  for i in range(len(Xtrain)):
    temp = []
    for j in Vocabword:
      temp.append(Xtrain[i].count(j))
    NewXtrain.append(temp)

  NewXtest = []
  for i in range(len(Xtest)):
    temp = []
    for j in Vocabword:
      temp.append(Xtest[i].count(j))
    NewXtest.append(temp)

  NewXtrain = np.array(NewXtrain)
  NewXtest = np.array(NewXtest)

  return NewXtrain,NewXtest,Vocabword

NewXtrain,NewXtest,Vocabword = FeatureMatrix(Xtrain, Xtest)

print('Size of vocabulary :',len(Vocabword))
print('Size of Feature Matrix for training data',NewXtrain.shape)
print('Size of Feature Matrix for testing data',NewXtest.shape)


Size of vocabulary : 1574
Size of Feature Matrix for training data (700, 1574)
Size of Feature Matrix for testing data (300, 1574)


In [18]:
#2.4.
# mnv = MultinomialNB(alpha = 1 ,fit_prior = False)
# mnv.fit(NewXtrain, Ytrain)
# path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_4/Saved_Models/2/'
# with open(path+'textclassification.pkl', "wb") as f: 
#   pk.dump(mnv, f)
# print(mnv)

MultinomialNB(alpha=1, class_prior=None, fit_prior=False)


In [20]:
#2.5.

def getresults(path,NewXtrain, NewXtest, Ytrain, Ytest):
  with open(path+'textclassification.pkl', "rb") as f: 
    mnv = pk.load(f)
  Ypredtrain = mnv.predict(NewXtrain)
  Ypredtest = mnv.predict(NewXtest)
  print('Training Accuracy',round(accuracy_score(Ytrain,Ypredtrain)*100,3),'%')
  print('Validation Accuracy',round(accuracy_score(Ytest,Ypredtest)*100,3),'%')
  print('Mis classified Instances :')

  k = 0
  i = 0
  Ytest = list(Ytest)
  Ypredtest = list(Ypredtest)
  output = []
  while(k<4):
    if(Ytest[i] != Ypredtest[i]):
      k = k+1
      output.append([Xtest[i],Ytest[i],Ypredtest[i]])
    i = i+1  

  Output = pd.DataFrame(output,columns=['Line','Actual Label','Predicted Label'])
  print(tabulate(Output, tablefmt = 'psql',headers = 'keys'))  

path = '/content/drive/My Drive/Projects/ML_Assignment/ML_Assignment_4/Saved_Models/2/'
getresults(path,NewXtrain, NewXtest, Ytrain, Ytest)  

Training Accuracy 96.571 %
Validation Accuracy 81.0 %
Mis classified Instances :
+----+-------------------------------------------------------------+----------------+-------------------+
|    | Line                                                        |   Actual Label |   Predicted Label |
|----+-------------------------------------------------------------+----------------+-------------------|
|  0 | ['rotating', 'beers', 'tap', 'also', 'highlight', 'place']  |              1 |                 0 |
|  1 | ['host', 'staff', 'lack', 'better', 'word', 'bitches']      |              0 |                 1 |
|  2 | ['pizza', 'tasted', 'old', 'super', 'chewy', 'good', 'way'] |              0 |                 1 |
|  3 | ['attached', 'gas', 'station', 'rarely', 'good', 'sign']    |              0 |                 1 |
+----+-------------------------------------------------------------+----------------+-------------------+
